## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import ast
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date  month_1  \
0        1     68646      10  1381620027  2013-10-12 23:20:27        0   
1        1    113277      10  1379466669  2013-09-18 01:11:09        0   
2        2    422720       8  1412178746  2014-10-01 15:52:26        0   
3        2    454876       8  1394818630  2014-03-14 17:37:10        0   
4        2    790636       7  1389963947  2014-01-17 13:05:47        0   

   month_2  month_3  month_4  month_5  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0         0          1  

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [2]:
user_items = reviews[['user_id', 'movie_id', 'rating']].astype({'user_id': 'uint32', 'movie_id': 'uint32', 'rating': 'uint8'})
user_items.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


In [20]:
user_items[user_items.user_id == 2].movie_id.to_list()

[422720,
 454876,
 790636,
 816711,
 1091191,
 1103275,
 1322269,
 1390411,
 1398426,
 1431045,
 1433811,
 1454468,
 1535109,
 1675434,
 1798709,
 2017038,
 2024544,
 2294629,
 2361509,
 2381249,
 2726560,
 2883512,
 3079380]

In [21]:
movies[movies.movie_id.isin(user_items[user_items.user_id == 2].movie_id.to_list())]

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
13196,422720,Marie Antoinette (2006),Biography|Drama|History,2006,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13676,454876,Life of Pi (2012),Adventure|Drama|Fantasy,2012,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14545,790636,Dallas Buyers Club (2013),Biography|Drama,2013,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14754,816711,World War Z (2013),Action|Adventure|Horror,2013,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
15885,1091191,Lone Survivor (2013),Action|Biography|Drama,2013,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
15946,1103275,Two Lovers (2008),Drama|Romance,2008,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17094,1322269,August: Osage County (2013),Drama,2013,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17408,1390411,In the Heart of the Sea (2015),Action|Adventure|Biography,2015,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
17447,1398426,Straight Outta Compton (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17622,1431045,Deadpool (2016),Action|Adventure|Sci-Fi,2016,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [3]:
# Create user-by-item matrix using multi index and unstack
user_by_items = user_items.set_index(['user_id', 'movie_id']).unstack()
user_by_items.columns = user_by_items.columns.droplevel(0)
print(user_by_items.info())
user_by_items.head()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 53968 entries, 1 to 53968
Columns: 31245 entries, 8 to 8439854
dtypes: float64(31245)
memory usage: 12.6 GB
None


movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#create dtype dictionary
dtype_dict={}
for col in user_by_items.columns:
    dtype_dict[col]='uint32'
user_by_items = user_by_items.astype(dtype_dict,errors='ignore')

In [9]:
print(user_by_items.info())
user_by_items.head()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 53968 entries, 1 to 53968
Columns: 31245 entries, 8 to 8439854
dtypes: float64(31245)
memory usage: 12.6 GB
None


movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check your results below to make sure your matrix is ready for the upcoming sections.

In [4]:
assert reviews.user_id.nunique() == user_by_items.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_items.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [9]:
user_items[user_items.user_id == 0 ]['movie_id'].values.tolist()

array([], dtype=uint32)

In [12]:
user_by_items.index

UInt64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
                10,
             ...
             53959, 53960, 53961, 53962, 53963, 53964, 53965, 53966, 53967,
             53968],
            dtype='uint64', name='user_id', length=53968)

In [13]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    # Implement your code here
    #movies = user_by_items.loc[user_id].dropna().index.to_list()
    movies = user_items[user_items.user_id == user_id ]['movie_id'].values.tolist()
    return movies


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    # Implement your code here
    movies_seen = {}
    for user_id in user_by_items.index :
        movies_seen[user_id] = movies_watched(user_id)
    return movies_seen

movies_seen = create_user_movie_dict()

In [16]:
movies_seen

{1: [68646, 113277],
 2: [422720,
  454876,
  790636,
  816711,
  1091191,
  1103275,
  1322269,
  1390411,
  1398426,
  1431045,
  1433811,
  1454468,
  1535109,
  1675434,
  1798709,
  2017038,
  2024544,
  2294629,
  2361509,
  2381249,
  2726560,
  2883512,
  3079380],
 3: [1790864, 2170439, 2203939],
 4: [1300854],
 5: [54953, 120863],
 6: [2103281],
 7: [1764234, 1790885, 2053463],
 8: [385002, 1220198, 1462900, 1512685, 1631707, 1986994, 1999995],
 9: [65207, 363163, 985699],
 10: [1253863],
 11: [3294634],
 12: [1255953],
 13: [2649554],
 14: [452623],
 15: [1980929],
 16: [800241],
 17: [68646, 71562, 99674],
 18: [2258858],
 19: [390384],
 20: [1549589, 1605783],
 21: [4094724],
 22: [359950,
  770828,
  831387,
  1091191,
  1170358,
  1210819,
  1430132,
  1490017,
  1535108,
  1663662,
  1843866,
  1872181,
  2103281,
  2179136,
  2209764,
  2267998,
  2333784,
  2334879,
  2713180,
  2872732],
 23: [2674426],
 24: [816692, 2245084, 2267998, 2488496, 2802144, 2820852, 33005

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [22]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    # Implement your code here
    movies_to_analyze = {}
    for user_id in movies_seen:
        if len(movies_seen[user_id]) > lower_bound:
            movies_to_analyze[user_id] = movies_seen[user_id]
    return movies_to_analyze

movies_to_analyze = create_movies_to_analyze(movies_seen)

In [27]:
len(movies_seen) - len(movies_to_analyze)

30456

In [31]:
len(movies_to_analyze[2])

23

In [32]:
# # Run the tests below to check that your movies_to_analyze matches the solution
# assert len(movies_to_analyze) == 3494, "Oops!  It doesn't look like your dictionary has the right number of individuals."
# assert len(movies_to_analyze[2]) == 10, "Oops!  User 2 didn't match the number of movies we thought they would have."
# assert len(movies_to_analyze[7])  == 10, "Oops!  User 7 didn't match the number of movies we thought they would have."
# print("If this is all you see, you are good to go!")

### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users. Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [126]:
user_by_items.loc[2].dropna().index.intersection(user_by_items.loc[104].dropna().index)

UInt64Index([454876, 816711, 1454468, 1535109], dtype='uint64', name='movie_id')

In [155]:
user_by_items.loc[[2,104],user_by_items.loc[2].dropna().index.intersection(user_by_items.loc[104].dropna().index)].T.corr()#.iloc[0,1]

user_id,2,104
user_id,,
2,NaN,NaN
104,NaN,1.0


In [152]:
np.corrcoef(user_by_items.loc[2][[454876, 816711, 1454468, 1535109]].values,user_by_items.loc[104][[454876, 816711, 1454468, 1535109]].values)

C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\ahmed\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


array([[nan, nan],
       [nan,  1.]])

In [149]:
user_by_items.loc[2][[454876, 816711, 1454468, 1535109]].cov(user_by_items.loc[104][[454876, 816711, 1454468, 1535109]])

0.0

In [120]:
np.intersect1d(user_by_items.loc[2], user_by_items.loc[104],assume_unique=False)

array([5., 7., 8., 9.])

In [156]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    # Implement your code here
    #find intersect movie
    corr = user_by_items.loc[[user1,user2],
                user_by_items.loc[user1].dropna().index.intersection(user_by_items.loc[user2].dropna().index)].\
                T. \
                corr().\
                iloc[0,1]
    return corr #return the correlation

In [157]:
compute_correlation(2,104)

nan

In [158]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(35,51), 2) == 0.84, "Oops!  The correlation between user 25 and 51 should be about 0.84."
assert np.isnan(compute_correlation(7,8022)), "Oops!  The correlation between user 7 and 8022 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

AssertionError: Oops!  The correlation between user 25 and 51 should be about 0.84.

### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 7 and 8022 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.

```
In the denominator of the correlation coefficient, we calculate the standard deviation for each user's ratings.  The ratings for two users share the same ratings on the matching movies.  Therefore, the standard deviation is 0.  Because a 0 is in the denominator of the correlation coefficient, we end up with a **NaN** correlation coefficient.  Therefore, a different approach is likely better for this particular situation.
```

In [52]:
# Which movies did both user 2 and user 104 see?
print(movies_seen[2])
print(movies_seen[104])


[422720, 454876, 790636, 816711, 1091191, 1103275, 1322269, 1390411, 1398426, 1431045, 1433811, 1454468, 1535109, 1675434, 1798709, 2017038, 2024544, 2294629, 2361509, 2381249, 2726560, 2883512, 3079380]
[356150, 359950, 364725, 454876, 463985, 770828, 800369, 816711, 831387, 903624, 942385, 1049413, 1210819, 1213663, 1234721, 1245492, 1300854, 1335975, 1343092, 1351685, 1408101, 1418377, 1428538, 1453405, 1454468, 1457767, 1499658, 1535108, 1535109, 1538403, 1549920, 1583421, 1591095, 1596343, 1623205, 1650554, 1663662, 1690953, 1706620, 1717715, 1840309, 1843866, 1854564, 1893256, 1899353, 1905041, 1924429, 1939659, 1951264, 1981115, 2024469, 2103281, 2234155, 2294449]


In [59]:
common_movie = []
for movie in movies_seen[2] :
    if movie in movies_seen[104]:
        common_movie.append(movie)
common_movie

[454876, 816711, 1454468, 1535109]

In [67]:
# What were the ratings for each user on those movies?
reviews[reviews.movie_id.isin(common_movie) & reviews.user_id.isin([2,104])].sort_values(['user_id','movie_id'])[['user_id','movie_id','rating']]

,user_id,movie_id,rating
3,2,454876,8
5,2,816711,8
13,2,1454468,8
14,2,1535109,8
1261,104,454876,9
1265,104,816711,7
1282,104,1454468,7
1286,104,1535109,9


`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [277]:
5 - np.NaN 

nan

In [159]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    # Implement your code here
    dist = np.linalg.norm((user_by_items.loc[user1] - user_by_items.loc[user2]).dropna())
    if np.isnan(dist):
        dist = 0
    return dist #return the euclidean distance

In [161]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == 0.0, "Oops!  The distance between a user and itself should be 0.0."
assert round(compute_euclidean_dist(35,51), 2) == 3.32, "Oops!  The distance between user 2 and 66 should be about 2.24."

print("If this is all you see, then it looks like your function passed all of our tests!")

AssertionError: Oops!  The distance between user 2 and 66 should be about 2.24.

In [164]:
# Don't change the code here
# This loads our solution dataframe with every possible user pairings
df_dists = pd.read_pickle('dists.p')
df_dists.dropna(inplace=True)

In [304]:
df_dists

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000
...,...,...,...
4719689,53966,53958,6.633250
4719690,53966,53959,5.291503
4719691,53966,53961,3.741657
4719692,53966,53964,13.304135


In [307]:
df_dists =df_dists.drop(df_dists[df_dists.user2 == df_dists.user1].index)

In [309]:
df_dists[df_dists.user2 == df_dists.user1]

,user1,user2,eucl_dist


### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [314]:
find_closest_neighbors(50)

array([22915, 34706, 33207, ..., 36807, 32494, 52737], dtype=int64)

In [335]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # Implement your code here
    closest_neighbors =df_dists[df_dists['user1']==user ][['user2','eucl_dist']].sort_values(by='eucl_dist')['user2'].values
    return closest_neighbors
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    # Implement your code here
    movies_liked = reviews[(reviews['user_id']==user_id) & (reviews.rating>= min_rating)]['movie_id'].values

    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids    
    '''
    # Implement your code here
    movie_lst = movies[movies.movie_id.isin(movie_ids)]['movie'].values
    return movie_lst
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    # Implement your code here
    closest = find_closest_neighbors(user)
    moviesLikes = set()
    for usr in closest:
        moviesLikes.update(set(movie_names( movies_liked(usr))))
        if len(moviesLikes)>=num_recs:
            break

    return list(moviesLikes)[:num_recs]
    #another solution but sort with distanse then by rating
    
    # movie_id =  reviews[reviews.user_id != user].set_index('user_id').join(closest, how='inner')[['movie_id','rating','eucl_dist']].\
    # sort_values(['eucl_dist','rating'],ascending=[True,False]).drop_duplicates(subset='movie_id',keep='first')['movie_id'].values
    # recommendations = movie_names(movie_id[:num_recs])
    # return recommendations

def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    # Implement your code here
    all_recs = {}
    for usr in list(movies_to_analyze.keys())[:1677]:
        all_recs[usr] = make_recommendations(usr,num_recs)
    return all_recs

all_recs = all_recommendations(10)

In [336]:
make_recommendations(2)

['Tomb Raider (2018)',
 'Le moine (2011)',
 'Cloudy with a Chance of Meatballs 2 (2013)',
 'Easy A (2010)',
 'Saving Mr. Banks (2013)',
 'Before Sunrise (1995)',
 'As Good as It Gets (1997)',
 'Miracle on 34th Street (1947)',
 'Sherlock Jr. (1924)',
 'The Raven (2012)']

In [331]:
all_recs 

{2: ['Teenage Mutant Ninja Turtles: Out of the Shadows (2016)',
  'Eye in the Sky (2015)',
  'Cloudy with a Chance of Meatballs 2 (2013)',
  'Coco (2017)',
  'Pandorum (2009)',
  'The Shallows (2016)',
  'Jûsan-nin no shikaku (2010)',
  'Die Welle (2008)',
  'Happythankyoumoreplease (2010)',
  'As Good as It Gets (1997)',
  'Sinister (2012)',
  'Hot Tub Time Machine 2 (2015)',
  'Song for Marion (2012)',
  'Stronger (2017)',
  'Liar Liar (1997)',
  'Water for Elephants (2011)',
  'Taste of Fear (1961)',
  'Collateral (2004)',
  'Shattered Glass (2003)',
  'Screamers (1995)',
  'Reservoir Dogs (1992)',
  'Toy Story (1995)',
  'Chef (2014)',
  'Trumbo (2015)',
  'The Book of Life (2014)',
  'About Schmidt (2002)',
  "God's Pocket (2014)",
  'Odd Thomas (2013)',
  'Ping Pong Summer (2014)',
  'White House Down (2013)',
  'Eddie the Eagle (2016)',
  'Meari to majo no hana (2017)',
  'Cut Bank (2014)',
  'Collateral Beauty (2016)',
  'Bajrangi Bhaijaan (2015)',
  'American Ultra (2015)',
  

In [337]:
assert all_recs[2] == make_recommendations(2), "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs[39] == make_recommendations(39), "Oops!  It actually wasn't possible to make any recommendations for user 39."
assert all_recs[1677] == make_recommendations(1677), "Oops! Looks like your solution for user 1677 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

If you made it here, you now have recommendations for many users using collaborative filtering!


### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [ ]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': # enter a letter,
    'The two methods used to estimate user similarity were: ': # enter a letter,
    'There was an issue with using the correlation coefficient.  What was it?': # enter a letter
}

t.test_recs(sol_dict)

In [ ]:
# Use the cells below for any work you need to do!

In [ ]:
# Users without recs


In [ ]:
# NaN euclidean distance values


In [ ]:
# Users with fewer than 10 recs
